Perfekt, dann schließen wir den API-Teil mit dem entscheidenden Thema der Fehlerbehandlung ab. Hier ist das Skript für den letzten Block.

-----

### Kapitel 6: Der Sicherheitsgurt für APIs – Robuste Clients und Fehlerbehandlung

# Kapitel 6: Der Sicherheitsgurt für APIs – Robuste Clients und Fehlerbehandlung

Bisher sind wir davon ausgegangen, dass unsere Anfragen immer erfolgreich sind. Das ist die "Schönwetter-Programmierung". In der Realität ist das Netzwerk jedoch unzuverlässig: Server können ausfallen, das Internet kann unterbrochen sein, APIs können langsam sein oder wir können eine fehlerhafte Anfrage senden. Ein professioneller Client stürzt in diesen Fällen nicht ab. Er fängt Fehler elegant ab und reagiert angemessen darauf.

## Wenn Dinge schiefgehen: Fehler elegant abfangen

### 6.1 Die Realität: Nichts funktioniert immer

Bei der Netzwerkkommunikation gibt es zwei grundlegend verschiedene Arten von Fehlern, die wir behandeln müssen:

1.  **Netzwerkfehler:** Das Problem tritt auf, bevor wir überhaupt eine Antwort vom Server erhalten. Unser Computer konnte den Server nicht erreichen.

      * Beispiele: Keine Internetverbindung, die Domain existiert nicht (DNS-Fehler), der Server antwortet nicht (Timeout).
      * Behandlung in Python: Diese Fehler lösen **Exceptions** im `requests`-Modul aus, die wir mit `try-except` fangen müssen.

2.  **HTTP-Fehler (Anwendungsfehler):** Die Verbindung war erfolgreich, und der Server hat uns geantwortet, aber seine Antwort signalisiert ein Problem.

      * Beispiele: Die angeforderte Seite wurde nicht gefunden (`404`), wir haben keine Berechtigung (`401`), oder auf dem Server selbst ist ein Fehler aufgetreten (`500`).
      * Behandlung in Python: Diese Fehler lösen **standardmäßig keine Exception** aus. Wir müssen den `response.status_code` manuell prüfen.

### 6.2 Netzwerkfehler abfangen mit `try-except`

Die `requests`-Bibliothek stellt eine Reihe von spezifischen Exceptions bereit, die wir abfangen können. Sie erben alle von der Basis-Exception `requests.exceptions.RequestException`.

**Die wichtigsten Exceptions:**

  * `requests.exceptions.ConnectionError`: Für allgemeine Netzwerkprobleme wie einen DNS-Fehler oder eine abgelehnte Verbindung.
  * `requests.exceptions.Timeout`: Wird ausgelöst, wenn der Server nicht innerhalb einer bestimmten Zeitspanne antwortet.

**Das `timeout`-Argument: Eine unverzichtbare Absicherung**
Standardmäßig wartet `requests` unbegrenzt auf eine Antwort. Wenn der Server hängt, hängt auch unser Programm. Daher ist es eine absolute Best Practice, **immer** ein `timeout`-Argument zu setzen.

`response = requests.get(url, timeout=5)`

Der Wert `5` bedeutet, dass `requests` maximal 5 Sekunden auf eine Antwort wartet. Dauert es länger, wird eine `Timeout`-Exception ausgelöst.

### 6.3 HTTP-Fehler systematisch behandeln

Nachdem wir sichergestellt haben, dass eine Antwort zurückkam, müssen wir ihren `status_code` prüfen. Eine `if/elif/else`-Struktur ist hierfür ideal.

```python
if response.status_code == 200:
    print("Alles super!")
elif response.status_code == 404:
    print("Ressource nicht gefunden.")
elif 400 <= response.status_code < 500:
    print(f"Client-Fehler: {response.status_code}")
elif 500 <= response.status_code < 600:
    print(f"Server-Fehler: {response.status_code}")
```

**Die Abkürzung: `response.raise_for_status()`**
Statt die `if/elif`-Kette manuell zu schreiben, können wir die Methode `.raise_for_status()` verwenden.

  * Wenn der Status-Code ein Erfolg ist (z.B. 200), tut die Methode **nichts**.
  * Wenn der Status-Code ein HTTP-Fehler ist (4xx oder 5xx), löst die Methode eine `requests.exceptions.HTTPError`-Exception aus.

Das erlaubt uns, sowohl Netzwerk- als auch HTTP-Fehler in einem einzigen, sauberen `try-except`-Block zu behandeln.

-----

## Fehlerbehandlung in der Praxis

### Beispiel 1: Einen Timeout abfangen

Wir versuchen, einen Endpunkt aufzurufen, der 3 Sekunden für eine Antwort braucht, setzen unser Timeout aber auf nur 1 Sekunde.

```python
import requests

# Dieser Endpunkt wartet 3 Sekunden, bevor er antwortet.
url = "http://httpbin.org/delay/3"

try:
    print("Sende Anfrage mit 1s Timeout...")
    # Die Anfrage wird fehlschlagen, da 1s < 3s ist.
    response = requests.get(url, timeout=1)
except requests.exceptions.Timeout:
    print("Fehler: Die Anfrage hat zu lange gedauert (Timeout).")
except requests.exceptions.RequestException as e:
    # Fängt alle anderen requests-Fehler ab.
    print(f"Ein anderer Fehler ist aufgetreten: {e}")
```

### Beispiel 2: Einen Verbindungsfehler abfangen

Wir versuchen, eine Domain aufzurufen, die nicht existiert. Dies wird einen `ConnectionError` auslösen.

```python
import requests

# Diese Domain existiert nicht.
url = "http://eine-domain-die-es-nicht-gibt.com"

try:
    print(f"Sende Anfrage an {url}...")
    response = requests.get(url)
except requests.exceptions.ConnectionError:
    print("Fehler: Konnte keine Verbindung herstellen. Überprüfe die URL und deine Internetverbindung.")
```

### Beispiel 3: `raise_for_status()` im Einsatz

Dies ist das empfohlene, moderne Muster. Wir kombinieren `try-except` mit `.raise_for_status()`, um alle Fehlerarten abzudecken.

```python
import requests

# Diese URL führt zu einem 404 Not Found Fehler.
url = "https://jsonplaceholder.typicode.com/posts/9999"

try:
    # Wir setzen ein vernünftiges Timeout.
    response = requests.get(url, timeout=5)
    
    # Prüfe auf HTTP-Fehler (4xx or 5xx) und löse ggf. eine Exception aus.
    response.raise_for_status()

    # Dieser Code wird nur bei Erfolg (Status 2xx) erreicht.
    print("Anfrage erfolgreich! Verarbeite Daten...")
    data = response.json()
    print(data)

except requests.exceptions.Timeout:
    print("Fehler: Timeout - Der Server hat nicht rechtzeitig geantwortet.")
except requests.exceptions.HTTPError as err:
    # Fängt speziell die Fehler von raise_for_status().
    print(f"HTTP-Fehler aufgetreten: {err}")
except requests.exceptions.RequestException as err:
    # Fängt alle anderen Netzwerk- und Verbindungsfehler ab.
    print(f"Ein Fehler ist aufgetreten: {err}")
```

-----

## Das Projekt wird kugelsicher: Robuster Buch-Client

Wir fassen die Logik unseres Buch-Clients in einer Funktion zusammen und bauen eine komplette, robuste Fehlerbehandlung darum herum. Dies ist die Blaupause für professionellen Client-Code.

```python
import requests

def fetch_book_data(isbn):
    """
    Ruft Buchdaten für eine gegebene ISBN ab.
    Enthält eine robuste Fehlerbehandlung für Netzwerk- und HTTP-Fehler.
    Gibt die Buchdaten als Dictionary bei Erfolg zurück, sonst None.
    """
    base_url = "https://openlibrary.org/api/books"
    params = {
        "bibkeys": f"ISBN:{isbn}",
        "format": "json",
        "jscmd": "data"
    }
    
    try:
        # Sende die Anfrage mit einem Timeout von 5 Sekunden.
        response = requests.get(base_url, params=params, timeout=5)
        
        # Prüfe auf HTTP-Fehler wie 404 oder 500.
        response.raise_for_status()
        
        data = response.json()
        
        # Manchmal gibt die API ein leeres Dictionary zurück, das ist auch ein Fehler.
        if not data:
            print(f"Fehler: Keine Daten für ISBN {isbn} gefunden.")
            return None
            
        # Extrahiere die relevanten Daten.
        book_key = f"ISBN:{isbn}"
        title = data[book_key]["title"]
        author = data[book_key]["authors"][0]["name"]
        
        return {"title": title, "author": author}

    except requests.exceptions.Timeout:
        print("Fehler: Timeout beim Abrufen der Buchdaten.")
        return None
    except requests.exceptions.HTTPError as e:
        print(f"HTTP-Fehler: {e.response.status_code} - Die Ressource konnte nicht abgerufen werden.")
        return None
    except (requests.exceptions.RequestException, KeyError) as e:
        # Fängt andere Netzwerkfehler oder Fehler bei der JSON-Verarbeitung (KeyError).
        print(f"Ein allgemeiner Fehler ist aufgetreten: {e}")
        return None


# --- Hauptprogramm ---
# Erfolgreicher Fall
clean_code_data = fetch_book_data("9780132350884")
if clean_code_data:
    print("\n--- Erfolgreicher Abruf ---")
    print(f"Titel: {clean_code_data['title']}, Autor: {clean_code_data['author']}")

print("\n" + "="*30 + "\n")

# Fehlerfall (nicht existierende ISBN)
invalid_book_data = fetch_book_data("123456789")
if not invalid_book_data:
    print("--- Fehlerfall wie erwartet ---")
    print("Der Abruf für die ungültige ISBN ist wie erwartet fehlgeschlagen.")

```

-----

### Deine Werkstatt: Übungen zu Kapitel 6

# Übungs-Set 6: Robuste API-Clients bauen

**Ziel:** In diesen Übungen lernen Sie, Ihre API-Clients so zu schreiben, dass sie nicht abstürzen, sondern sinnvoll auf verschiedene Fehlersituationen reagieren.

### Aufgabe 1: Gezielter 404-Fehler

Senden Sie eine `GET`-Anfrage an `https://jsonplaceholder.typicode.com/posts/9999` (dieser Post existiert nicht). Schreiben Sie eine `if`-Abfrage, die den `status_code` prüft und eine benutzerfreundliche Nachricht `"Der angeforderte Post konnte nicht gefunden werden."` ausgibt, falls der Code `404` ist.

### Aufgabe 2: Einen Timeout provozieren

Senden Sie eine `GET`-Anfrage an den Endpunkt `http://httpbin.org/delay/5`, der 5 Sekunden für eine Antwort benötigt. Setzen Sie in Ihrer Anfrage aber ein `timeout` von nur 2 Sekunden. Fangen Sie die `requests.exceptions.Timeout`-Exception mit einem `try-except`-Block ab und geben Sie eine passende Fehlermeldung aus.

### Aufgabe 3: Funktion mit `raise_for_status()`

Schreiben Sie eine Funktion `get_url(url)`, die eine URL als Argument nimmt.

  * Innerhalb der Funktion soll ein `try-except`-Block stehen.
  * Im `try`-Teil soll die URL mit `requests.get()` aufgerufen werden und direkt danach `response.raise_for_status()`.
  * Bei Erfolg soll die Funktion das `response`-Objekt zurückgeben.
  * Im `except`-Teil sollen `requests.exceptions.RequestException`-Fehler abgefangen und eine Fehlermeldung ausgegeben werden, woraufhin die Funktion `None` zurückgibt.
    Testen Sie Ihre Funktion mit einer gültigen und einer ungültigen URL.

### Aufgabe 4: **Schüler-Projekt: Robuster Post-Abruf**

Erweitern Sie Ihr Skript aus der letzten Übung, das einen einzelnen Post von JSONPlaceholder abruft.

1.  Kapseln Sie die Logik in einer Funktion `get_post_by_id(post_id)`.
2.  Integrieren Sie eine vollständige Fehlerbehandlung in diese Funktion:
      * Fangen Sie Netzwerkfehler (`ConnectionError`, `Timeout`) ab.
      * Prüfen Sie auf einen `404`-Status-Code, falls der Post nicht existiert.
3.  Die Funktion soll bei Erfolg das Post-Dictionary zurückgeben und in allen Fehlerfällen `None`.
4.  Rufen Sie die Funktion im Hauptteil Ihres Skripts mit einer gültigen ID (z.B. `1`) und einer ungültigen ID (z.B. `200`) auf und geben Sie das Ergebnis entsprechend aus.

### Aufgabe 5: **Schüler-Projekt: Fehlerbehandlung beim Erstellen**

Nehmen Sie Ihr Skript, das per `POST` einen neuen Post erstellt. Bauen Sie einen `try-except`-Block um den `requests.post()`-Aufruf, um Netzwerkfehler abzufangen. Geben Sie im Fehlerfall eine Nachricht aus wie "Der neue Post konnte nicht erstellt werden. Bitte versuchen Sie es später erneut."

### Aufgabe 6 (Challenge): Umgang mit Ratenbegrenzung

Die GitHub-API (`https://api.github.com`) hat eine öffentliche Ratenbegrenzung. Wenn man zu viele Anfragen in kurzer Zeit stellt, antwortet sie mit dem Status-Code `403 Forbidden`.
Schreiben Sie ein Skript, das in einer `while True`-Schleife Anfragen an `https://api.github.com/users/google` sendet. Innerhalb der Schleife:

1.  Prüfen Sie den `status_code` der Antwort.
2.  Wenn der Code `403` ist, geben Sie eine Meldung "Ratenbegrenzung erreicht. Breche ab." aus und beenden Sie die Schleife mit `break`.
3.  Andernfalls geben Sie "Anfrage erfolgreich..." aus.

### Aufgabe 7 (Challenge): Authentifizierungsfehler simulieren

Viele APIs erfordern einen API-Schlüssel, der in den Headern gesendet wird, z.B. `{'Authorization': 'Bearer MEIN_GEHEIMER_SCHLÜSSEL'}`. Wenn der Schlüssel falsch ist, antwortet der Server oft mit `401 Unauthorized`.
Schreiben Sie eine Funktion, die eine `GET`-Anfrage an `https://httpbin.org/status/401` sendet. Dieser Endpunkt simuliert eine `401`-Antwort. Ihre Funktion soll diesen spezifischen Status-Code erkennen und eine benutzerfreundliche Meldung ausgeben: "Fehler: Authentifizierung fehlgeschlagen. Überprüfen Sie Ihren API-Schlüssel."